# Importing Necessary Libraries

In [ ]:
%pip install pymc3

In [ ]:
import os
os.environ['MKL_THREADING_LAYER'] = 'GNU'

import pymc3 as pm
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix

# Data Preprocessing

In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/WillKoehrsen/eecs-491/master/assign/project/diabetes.csv')
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
data['Glucose'] = data['Glucose'].replace({0: data['Glucose'].median()})
data['BloodPressure'] = data['BloodPressure'].replace({0: data['BloodPressure'].median()})
data['SkinThickness'] = data['SkinThickness'].replace({0: data['SkinThickness'].median()})
data['Insulin'] = data['Insulin'].replace({0: data['Insulin'].median()})
data['BMI'] = data['BMI'].replace({0: data['BMI'].median()})

In [ ]:
features = data.drop(columns='Outcome')
labels = data.Outcome

# Split into training and testing set using 200 observations for testing
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state = 50)

print(X_train.shape)
print(X_test.shape)

(614, 8)
(154, 8)


In [ ]:
def corrupt_outcome_series(series, flip_ratio=0.1, random_state=None):
    # Make a copy of the Series to avoid modifying the original
    corrupted_series = series.copy()

    # Randomly select indices to flip
    np.random.seed(random_state)
    flip_indices = np.random.choice(series.index, size=int(len(series) * flip_ratio), replace=False)

    # Flip the values at the selected indices
    corrupted_series.loc[flip_indices] = 1 - corrupted_series.loc[flip_indices]

    return corrupted_series, flip_indices.tolist()

In [ ]:
corrupted_y_train, corrupted_indices = corrupt_outcome_series(y_train, flip_ratio=0.5, random_state=42)

print(len(corrupted_indices))

307


# Creating the Basic Model

In [ ]:
formula = [' %s + ' % variable for variable in X_train.columns]
formula.insert(0, 'y ~ ')
formula = ' '.join(''.join(formula).split(' ')[:-2])
formula

'y ~  Pregnancies +  Glucose +  BloodPressure +  SkinThickness +  Insulin +  BMI +  DiabetesPedigreeFunction +  Age'

In [ ]:
corrupted_X_with_labels = X_train.copy()
corrupted_X_with_labels['y'] = corrupted_y_train

fresh_X_with_labels = X_train.copy()
fresh_X_with_labels['y'] = y_train

In [ ]:
with pm.Model() as logistic_model:
    pm.GLM.from_formula(formula, data = corrupted_X_with_labels, family = pm.glm.families.Binomial())
    sampler = pm.NUTS()
    trace_log_corrupted = pm.sample(draws=500, step = sampler, chains=2, tune=1000, random_seed=100)

We recommend to instead use Bambi https://bambinos.github.io/bambi/
/usr/local/lib/python3.10/dist-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)


/usr/local/lib/python3.10/dist-packages/arviz/utils.py:187: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  numba_fn = numba.jit(**self.kwargs)(self.function)


In [ ]:
with pm.Model() as logistic_model:
    pm.GLM.from_formula(formula, data = fresh_X_with_labels, family = pm.glm.families.Binomial())
    sampler = pm.NUTS()
    trace_log_fresh = pm.sample(draws=500, step = sampler, chains=2, tune=1000, random_seed=100)

We recommend to instead use Bambi https://bambinos.github.io/bambi/
/usr/local/lib/python3.10/dist-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)


In [ ]:
def evaluate_trace(trace, data, print_model = False):
    means_dict = {}
    std_dict = {}

    for var in trace.varnames:
        means_dict[var] = np.mean(trace[var])
        std_dict[var] = np.std(trace[var])

    model = 'logit = %0.4f + ' % np.mean(means_dict['Intercept'])

    for var in data.columns:
        model += '%0.4f * %s + ' % (means_dict[var], var)

    model = ' '.join(model.split(' ')[:-2])
    if print_model:
        print('Final Equation: \n{}'.format(model))

    return means_dict, std_dict

In [ ]:
print(evaluate_trace(trace_log_fresh, X_test, True))
print(evaluate_trace(trace_log_corrupted, X_test, True))

Final Equation: 
logit = -9.3728 + 0.1303 * Pregnancies + 0.0438 * Glucose + -0.0237 * BloodPressure + -0.0071 * SkinThickness + -0.0025 * Insulin + 0.1198 * BMI + 0.9237 * DiabetesPedigreeFunction + 0.0136 * Age
({'Intercept': -9.372786605355405, 'Pregnancies': 0.13026566609553125, 'Glucose': 0.04377927885864726, 'BloodPressure': -0.02367874896481254, 'SkinThickness': -0.00707213329981328, 'Insulin': -0.0024633121681453102, 'BMI': 0.1198413888160493, 'DiabetesPedigreeFunction': 0.9236618179956936, 'Age': 0.013563921049604388}, {'Intercept': 0.9677676198152729, 'Pregnancies': 0.03912340385362138, 'Glucose': 0.004535930509215271, 'BloodPressure': 0.010529340882414131, 'SkinThickness': 0.015448461780700789, 'Insulin': 0.001086083974959748, 'BMI': 0.02170928807921105, 'DiabetesPedigreeFunction': 0.3490810947743894, 'Age': 0.01159125263435034})
Final Equation: 
logit = 0.1508 + 0.0301 * Pregnancies + 0.0022 * Glucose + 0.0026 * BloodPressure + -0.0195 * SkinThickness + 0.0010 * Insulin + 0

In [ ]:
def find_probs(trace, data):
    # Find the means and std of the variables
    means_dict, std_dict = evaluate_trace(trace, data)

    probs = []

    mean_array = np.array(list(means_dict.values()))

    # Need an intercept term in the data
    data['Intercept'] = 1
    data = data[list(means_dict.keys())]

    # Calculate the probability for each observation in the data
    for _, row in data.iterrows():
        # First the log odds
        logit = np.dot(row, mean_array)
        # Convert the log odds to a probability
        probability = 1 / (1 + np.exp(-logit))
        probs.append(probability)

    return probs

In [ ]:
def calc_metrics(predictions, y_test):
    accuracy = np.mean(predictions == y_test)
    f1_metric = f1_score(y_test, predictions)

    print('Accuracy of Model: {:.2f}%'.format(100 * accuracy))
    print('F1 Score of Model: {:.4f}'.format(f1_metric))

    return accuracy, f1_metric

In [ ]:
blr_probs = find_probs(trace_log_fresh, X_test)

predictions = (np.array(blr_probs) > 0.5)
calc_metrics(predictions, y_test)

Accuracy of Model: 73.38%
F1 Score of Model: 0.5495


(0.7337662337662337, 0.5494505494505494)

In [ ]:
blr_probs = find_probs(trace_log_corrupted, X_test)

predictions = (np.array(blr_probs) > 0.5)
calc_metrics(predictions, y_test)

Accuracy of Model: 54.55%
F1 Score of Model: 0.4262


(0.5454545454545454, 0.42622950819672134)

In [ ]:
failure_Xs = X_test[predictions != y_test]
failure_ys = y_test[predictions != y_test]

In [ ]:
F = failure_Xs.copy()
F['y'] = failure_ys

D_F = pd.concat([corrupted_X_with_labels, F])

In [ ]:
with pm.Model() as logistic_model:
    # Build the model using the formula and specify the data likelihood
    pm.GLM.from_formula(formula, data = D_F, family = pm.glm.families.Binomial())

    # Using the no-uturn sampler
    sampler = pm.NUTS()

    # Sample from the posterior using NUTS
    trace_log_failure_inc = pm.sample(draws=500, step = sampler, chains=2, tune=1000, random_seed=100)

We recommend to instead use Bambi https://bambinos.github.io/bambi/
/usr/local/lib/python3.10/dist-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)


In [ ]:
blr_probs = find_probs(trace_log_failure_inc, X_test)

predictions = (np.array(blr_probs) > 0.5)
calc_metrics(predictions, y_test)

Accuracy of Model: 72.08%
F1 Score of Model: 0.4691


(0.7207792207792207, 0.46913580246913583)

In [ ]:
def find_r_z(trace1, trace2, data):
    print(data.info())
    # Find the means and std of the variables
    means_dict_1, std_dict_1 = evaluate_trace(trace1, data)
    means_dict_2, std_dict_2 = evaluate_trace(trace2, data)

    probs = []

    mean_array_1 = np.array(list(means_dict_1.values()))
    mean_array_2 = np.array(list(means_dict_2.values()))

    # Need an intercept term in the data
    data['Intercept'] = 1
    data = data[list(means_dict_1.keys())]

    # Calculate the probability for each observation in the data
    for id, row in data.iterrows():
        # First the log odds
        logit_1 = np.dot(row, mean_array_1)
        logit_2 = np.dot(row, mean_array_2)
        # Convert the log odds to a probability
        probability_1 = 1 / (1 + np.exp(-logit_1))
        probability_2 = 1 / (1 + np.exp(-logit_2))

        probs.append((id, probability_1 - probability_2))

    return probs

In [ ]:
r_z = find_r_z(trace_log_corrupted, trace_log_failure_inc, X_train)

<class 'pandas.core.frame.DataFrame'>
Index: 614 entries, 720 to 688
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               614 non-null    int64  
 1   Glucose                   614 non-null    int64  
 2   BloodPressure             614 non-null    int64  
 3   SkinThickness             614 non-null    int64  
 4   Insulin                   614 non-null    float64
 5   BMI                       614 non-null    float64
 6   DiabetesPedigreeFunction  614 non-null    float64
 7   Age                       614 non-null    int64  
 8   Intercept                 614 non-null    int64  
dtypes: float64(3), int64(6)
memory usage: 48.0 KB
None


In [ ]:
filtered_list = [x for x in r_z if x[1] > 0]
indices_to_remove = [x[0] for x in filtered_list]
print(indices_to_remove)

[720, 691, 482, 222, 339, 23, 331, 282, 484, 234, 443, 715, 157, 646, 337, 264, 25, 503, 172, 530, 42, 488, 192, 168, 667, 85, 243, 252, 621, 52, 493, 124, 81, 464, 265, 624, 721, 433, 421, 137, 166, 457, 279, 368, 743, 98, 220, 696, 5, 465, 595, 24, 424, 729, 669, 145, 315, 553, 494, 611, 423, 324, 416, 249, 723, 149, 474, 65, 214, 604, 14, 182, 454, 742, 303, 468, 135, 97, 510, 515, 372, 511, 208, 317, 571, 524, 685, 613, 633, 555, 690, 298, 170, 102, 490, 268, 236, 383, 648, 543, 136, 727, 15, 290, 473, 333, 328, 257, 88, 126, 600, 80, 612, 736, 3, 280, 353, 247, 365, 708, 527, 72, 219, 660, 513, 745, 683, 637, 394, 615, 94, 609, 281, 744, 347, 276, 76, 190, 87, 694, 313, 75, 179, 628, 355, 525, 678, 650, 750, 391, 573, 194, 244, 200, 576, 675, 47, 610, 455, 283, 758, 535, 409, 709, 158, 614, 717, 629, 658, 430, 297, 119, 597, 9, 274, 32, 514, 259, 307, 360, 55, 547, 400, 765, 181, 89, 634, 338, 50, 44, 687, 392, 61, 84, 73, 554, 643, 325, 103, 113, 237, 528, 226, 574, 238, 306, 645

In [ ]:
co = 0

for m in filtered_list:
    if m[0] in corrupted_indices:
        co += 1

print(co/307)

0.6058631921824105


In [ ]:
filtered_corrupted_X_with_labels = corrupted_X_with_labels.drop(corrupted_indices)

In [ ]:
with pm.Model() as logistic_model:
    pm.GLM.from_formula(formula, data = filtered_corrupted_X_with_labels, family = pm.glm.families.Binomial())
    sampler = pm.NUTS()
    trace_log_filtered_X = pm.sample(draws=500, step = sampler, chains=2, tune=1000, random_seed=100)

We recommend to instead use Bambi https://bambinos.github.io/bambi/
/usr/local/lib/python3.10/dist-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)


In [ ]:
blr_probs = find_probs(trace_log_filtered_X, X_test)

predictions = (np.array(blr_probs) > 0.5)
calc_metrics(predictions, y_test)

Accuracy of Model: 70.13%
F1 Score of Model: 0.5106


(0.7012987012987013, 0.5106382978723404)